# 1d filter

In [81]:
import torch
from torch.autograd import Variable
from torch.nn import Conv1d,Sequential,Module
from torch.optim import Adam, SGD

In [82]:
torch.cuda.is_available()

True

## building a convolutional network 

1D CNN; filter has fixed weights generated by fortran code

In [83]:
chanl_in = 1 
chanl_out = 10 # n_loc
# our filter size is 10x1x10
# filter = a_filter -> returned by fortran code

class Net(Module):
    def __init__(self):
        super(Net, self).__init__() 

        self.cnn_layers = Sequential(
            Conv1d(chanl_in,chanl_out, kernel_size=10,bias=False)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), 1, -1)
        return x

In [84]:
torch.random.manual_seed(42)
net = Net()

input = torch.randn(1, 1, 10)


In [85]:
input = torch.ones(12, 1, 10)
print(input)
weights_layer1 = net.cnn_layers[0].weight.data
# print(weights_layer1)
print(weights_layer1.shape)
output = net(input)
# print(input)
print(output)
print(weights_layer1)
print(input.shape)
print(output.shape)
output=torch.unsqueeze(output,1)
print(output.shape)

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]])
torch.Size([10, 1, 10])
tensor([[[ 0.7937,  0.9418, -1.0588,  0.6461,  0.6662, -0.0789, -0.4787,
           0.1634, -0.3500, -0.2437]],

        [[ 0.7937,  0.9418, -1.0588,  0.6461,  0.6662, -0.0789, -0.4787,
           0.1634, -0.3500, -0.2437]],

        [[ 0.7937,  0.9418, -1.0588,  0.6461,  0.6662, -0.0789, -0.4787,
           0.1634, -0.3500, -0.2437]],

 

In [86]:
# filter from FEM assemble... (-dt M^-1 K + I)
A = [[      1.,          0,          0,         0,        0,        0,        0,        0,         0,        0],
[      0,          1.,          0,         0,        0,        0,        0,        0,         0,        0],
[      0,          0,          1.,         0,        0,        0,        0,        0,         0,        0],
[      0,          0,          0,         1.,        0,        0,        0,        0,         0,        0],
[      0,          0,          0,         0,        1.,        0,        0,        0,         0,        0],
[31./1000, -139./13500,     1./3375, -183./2000, 739./6000, 989./1200, -61./6000, -11./1200,  -93./2000, 189./1000],
[31./1000,     1./3375, -139./13500,  -93./2000, -11./1200, -61./6000, 989./1200, 739./6000, -183./2000, 189./1000],
[      0,          0,          0,         0,        0,        0,        0,        1.,         0,        0],
[      0,          0,          0,         0,        0,        0,        0,        0,         1.,        0],
[-7./1500,    -7./3000,    -7./3000,   49./1500,   7./3000,  91./3000,  91./3000,   7./3000,   49./1500,  659./750]]
A=torch.tensor(A)


In [87]:
print(A.shape)
A = torch.unsqueeze(A,1)
print(A.shape)

torch.Size([10, 10])
torch.Size([10, 1, 10])


In [88]:
net.cnn_layers[0].weight.data = A

In [89]:
net.cnn_layers[0].weight.data

tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 3.1000e-02, -1.0296e-02,  2.9630e-04, -9.1500e-02,  1.2317e-01,
           8.2417e-01, -1.0167e-02, -9.1667e-03, -4.6500e-02,  1.8900e-01]],

        [[ 3.1000e-02,  2.9630e-04, -1.0296e-02, -4.6500e-02, -9.1667e-03,
       

In [93]:
# initial condition
c_n_minus1=torch.tensor([[[0., 0., 1., 0., 0., 0., 0., 1., 1., 0.]]])
for i in range(2000):
    c_n = net.forward(c_n_minus1)
    c_n_minus1 = c_n 

In [94]:
print(c_n)

tensor([[[0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.1387, 0.6019, 1.0000,
          1.0000, 0.4544]]], grad_fn=<ViewBackward0>)
